In [ ]:
!pip install python-dotenv
!pip install langchain

In [7]:
!pip install langchain

  Using cached langchain-0.3.27-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_core-0.3.76-py3-none-any.whl.metadata (3.7 kB)
  Using cached langchain_text_splitters-0.3.11-py3-none-any.whl.metadata (1.8 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
Using cached langchain-0.3.27-py3-none-any.whl (1.0 MB)
Using cached langchain_core-0.3.76-py3-none-any.whl (447 kB)
Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
Using cached langchain_text_splitters-0.3.11-py3-none-any.whl (33 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.6 MB/s  0:00:00
Using cached tenacity-9.1.2-py3-none-any.whl (28 kB)
Using cached jsonpointer-3.0.0-py2.py3-none-any.whl

In [8]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

# Learning to have conversation with LLM

In [10]:
!pip install -U langchain-groq

  Using cached langchain_groq-0.3.8-py3-none-any.whl.metadata (2.6 kB)
  Using cached groq-0.31.1-py3-none-any.whl.metadata (16 kB)
Using cached langchain_groq-0.3.8-py3-none-any.whl (16 kB)
Using cached groq-0.31.1-py3-none-any.whl (134 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [langchain-groq]


In [11]:
# pick a model
from langchain.chat_models import init_chat_model
llm = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

### Create your tools

In [20]:
from langchain_core.tools import tool

@tool
def salty() -> str:
    """ Call this tool if the food is salty"""
    return "Its Salty"


@tool
def sour() -> str:
    """ Call this tool if the food is sour"""
    return "Its Sour"


@tool
def sweet() -> str:
    """ Call this tool if the food is sweet"""
    return "Its Sweet"

@tool
def bitter() -> str:
    """ Call this tool if the food is bitter"""
    return "Its Bitter"




In [21]:
tools_list = [bitter, sour, sweet, salty]
tools_dict = {t.name: t for t in tools_list} # comes in handy at the time of invokation

In [22]:
# we create a tool calling Agent by binding a list of tools to the llm
llm_with_tools = llm.bind_tools(tools_list)

In [23]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage

# This will store all converation
chat_history = [
    SystemMessage(content="You are a bot which can taste food, and also recommend kind of food the user is craving. Depending on the conversation, either respond the question, or show taste of the food/craving the user has with the situation")
    ]

In [27]:
chat_history.append(HumanMessage(content="I just had some French fries. What does it taste like?"))

response = llm_with_tools.invoke(chat_history)
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '9eakavyy2', 'function': {'arguments': 'null', 'name': 'salty'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 433, 'total_tokens': 441, 'completion_time': 0.038598624, 'prompt_time': 0.040754806, 'queue_time': 0.055474054, 'total_time': 0.07935343}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--a427857e-dda1-464f-b1d7-f2637d2ba070-0', tool_calls=[{'name': 'salty', 'args': {}, 'id': '9eakavyy2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 433, 'output_tokens': 8, 'total_tokens': 441})

In [28]:
chat_history.append(response)
chat_history

[SystemMessage(content='You are a bot which can taste food, and also recommend kind of food the user is craving. Depending on the conversation, either respond the question, or show taste of the food/craving the user has with the situation', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Your input is incomplete. Please provide more details or specify the food you'd like me to taste or recommend.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 388, 'total_tokens': 411, 'completion_time': 0.097957414, 'prompt_time': 0.033485955, 'queue_time': 0.052154645, 'total_time': 0.131443369}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_155ab82e98', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--0856438a-67c4-4227-afe1-aedb00b51251-0', usage_metadata={'input_tokens': 388, 'output_tokens': 23, 'total_to

In [29]:
# Since there was a tool call, execute the tool and append the tool output in the converation

chat_history.append(tools_dict[response.tool_calls[0]["name"]].invoke(response.tool_calls[0]))
chat_history

[SystemMessage(content='You are a bot which can taste food, and also recommend kind of food the user is craving. Depending on the conversation, either respond the question, or show taste of the food/craving the user has with the situation', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Your input is incomplete. Please provide more details or specify the food you'd like me to taste or recommend.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 388, 'total_tokens': 411, 'completion_time': 0.097957414, 'prompt_time': 0.033485955, 'queue_time': 0.052154645, 'total_time': 0.131443369}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_155ab82e98', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--0856438a-67c4-4227-afe1-aedb00b51251-0', usage_metadata={'input_tokens': 388, 'output_tokens': 23, 'total_to

In [30]:
chat_history.append(HumanMessage(content="This pretzel is making me thirsty... what taste is it?"))
response = llm_with_tools.invoke(chat_history)
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'm4e3zhh4c', 'function': {'arguments': '{}', 'name': 'salty'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 473, 'total_tokens': 482, 'completion_time': 0.02450578, 'prompt_time': 0.042696362, 'queue_time': 0.046168407, 'total_time': 0.067202142}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--aef6dc04-04bb-42e6-bdde-1f7cfb036e59-0', tool_calls=[{'name': 'salty', 'args': {}, 'id': 'm4e3zhh4c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 473, 'output_tokens': 9, 'total_tokens': 482})

In [31]:
chat_history.append(response)
chat_history

[SystemMessage(content='You are a bot which can taste food, and also recommend kind of food the user is craving. Depending on the conversation, either respond the question, or show taste of the food/craving the user has with the situation', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Your input is incomplete. Please provide more details or specify the food you'd like me to taste or recommend.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 388, 'total_tokens': 411, 'completion_time': 0.097957414, 'prompt_time': 0.033485955, 'queue_time': 0.052154645, 'total_time': 0.131443369}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_155ab82e98', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--0856438a-67c4-4227-afe1-aedb00b51251-0', usage_metadata={'input_tokens': 388, 'output_tokens': 23, 'total_to

In [32]:
chat_history.append(tools_dict[response.tool_calls[0]["name"]].invoke(response.tool_calls[0]))
chat_history

[SystemMessage(content='You are a bot which can taste food, and also recommend kind of food the user is craving. Depending on the conversation, either respond the question, or show taste of the food/craving the user has with the situation', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Your input is incomplete. Please provide more details or specify the food you'd like me to taste or recommend.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 388, 'total_tokens': 411, 'completion_time': 0.097957414, 'prompt_time': 0.033485955, 'queue_time': 0.052154645, 'total_time': 0.131443369}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_155ab82e98', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--0856438a-67c4-4227-afe1-aedb00b51251-0', usage_metadata={'input_tokens': 388, 'output_tokens': 23, 'total_to

In [33]:
chat_history.append(HumanMessage(content="But i added sugar to it"))
response = llm_with_tools.invoke(chat_history)
chat_history.append(response)
chat_history

[SystemMessage(content='You are a bot which can taste food, and also recommend kind of food the user is craving. Depending on the conversation, either respond the question, or show taste of the food/craving the user has with the situation', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Your input is incomplete. Please provide more details or specify the food you'd like me to taste or recommend.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 388, 'total_tokens': 411, 'completion_time': 0.097957414, 'prompt_time': 0.033485955, 'queue_time': 0.052154645, 'total_time': 0.131443369}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_155ab82e98', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--0856438a-67c4-4227-afe1-aedb00b51251-0', usage_metadata={'input_tokens': 388, 'output_tokens': 23, 'total_to

In [35]:
# Execute the tool, append the response
chat_history.append(tools_dict[response.tool_calls[0]["name"]].invoke(response.tool_calls[0]))
chat_history

[SystemMessage(content='You are a bot which can taste food, and also recommend kind of food the user is craving. Depending on the conversation, either respond the question, or show taste of the food/craving the user has with the situation', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Your input is incomplete. Please provide more details or specify the food you'd like me to taste or recommend.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 388, 'total_tokens': 411, 'completion_time': 0.097957414, 'prompt_time': 0.033485955, 'queue_time': 0.052154645, 'total_time': 0.131443369}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_155ab82e98', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--0856438a-67c4-4227-afe1-aedb00b51251-0', usage_metadata={'input_tokens': 388, 'output_tokens': 23, 'total_to

In [37]:
# lets see, if the model remebers the context
chat_history.append(HumanMessage(content="I’m stranded on a desert island and all I have are crackers. What taste will keep me alive?"))
response = llm_with_tools.invoke(chat_history)
chat_history.append(response)
chat_history

[SystemMessage(content='You are a bot which can taste food, and also recommend kind of food the user is craving. Depending on the conversation, either respond the question, or show taste of the food/craving the user has with the situation', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Your input is incomplete. Please provide more details or specify the food you'd like me to taste or recommend.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 388, 'total_tokens': 411, 'completion_time': 0.097957414, 'prompt_time': 0.033485955, 'queue_time': 0.052154645, 'total_time': 0.131443369}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_155ab82e98', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--0856438a-67c4-4227-afe1-aedb00b51251-0', usage_metadata={'input_tokens': 388, 'output_tokens': 23, 'total_to

In [38]:
chat_history.append(HumanMessage(content="A dragon offered me a lemon drop as a peace treaty. How does it taste?"))
response = llm_with_tools.invoke(chat_history)
chat_history.append(response)
chat_history

[SystemMessage(content='You are a bot which can taste food, and also recommend kind of food the user is craving. Depending on the conversation, either respond the question, or show taste of the food/craving the user has with the situation', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Your input is incomplete. Please provide more details or specify the food you'd like me to taste or recommend.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 388, 'total_tokens': 411, 'completion_time': 0.097957414, 'prompt_time': 0.033485955, 'queue_time': 0.052154645, 'total_time': 0.131443369}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_155ab82e98', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--0856438a-67c4-4227-afe1-aedb00b51251-0', usage_metadata={'input_tokens': 388, 'output_tokens': 23, 'total_to